# take as input TCGA illumina hiseq data, output countData and colData files for DESeq2

In [5]:
import pandas as pd

expr = pd.read_csv("data/UCEC.rnaseqv2__illuminahiseq_rnaseqv2__unc_edu__Level_3__RSEM_genes__data.data.txt",sep="\t",index_col=0)
expr = expr.iloc[1:].T.set_index(expr.iloc[0], append=True).T # multidex the dataframe
expr = expr.xs('raw_count', axis=1, level=1) # select columns that are raw_count measurements
expr = expr.astype(float).astype(int) # cast to float, then to int
expr

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,2,4,5,7,8,10,11,13,14,16,17,19,20,22,23,25,26,28,29,31,32,34,35,37,38,40,41,43,44,46,47,49,50,52,53,55,56,58,59,61,62,64,65,67,68,70,71,73,74,76,77,79,80,82,83,85,86,88,89,91,92,94,95,97,98,100,101,103,104,106,107,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224,226,227,229,230,232,233,235,236,238,239,241,242,244,245,247,248,250,251,253,254,256,257,259,260,262,263,265,266,268,269,271,272,274,275,277,278,280,281,283,284,286,287,289,290,292,293,295,296,298,299,301,302,304,305,307,308,310,311,313,314,316,317,319,320,322,323,325,326,328,329,331,332,334,335,337,338,340,341,343,344,346,347,349,350,352,353,355,356,358,359,361,362

,TCGA-2E-A9G8-01A-11R-A40A-07,TCGA-4E-A92E-01A-11R-A37O-07,TCGA-5B-A90C-01A-11R-A37O-07,TCGA-5S-A9Q8-01A-11R-A40A-07,TCGA-A5-A1OH-01A-21R-A22K-07,TCGA-A5-A2K2-01A-11R-A18M-07,TCGA-A5-A2K3-01A-11R-A32Y-07,TCGA-A5-A2K4-01A-11R-A18M-07,TCGA-A5-A2K5-01A-11R-A180-07,TCGA-A5-A2K7-01A-11R-A180-07,...,TCGA-QF-A5YS-01A-11R-A31O-07,TCGA-QF-A5YT-01A-11R-A31O-07,TCGA-QS-A5YQ-01A-11R-A31O-07,TCGA-QS-A5YR-01A-31R-A31O-07,TCGA-QS-A744-01A-11R-A34R-07,TCGA-QS-A8F1-01A-21R-A37O-07,TCGA-SJ-A6ZI-01A-12R-A34R-07,TCGA-SJ-A6ZJ-01A-12R-A34R-07,TCGA-SL-A6J9-01A-11R-A31O-07,TCGA-SL-A6JA-01A-11R-A31O-07
Hybridization REF,,,,,,,,,,,,,,,,,,,,,
?|100130426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
?|100133144,23,88,62,13,48,9,0,22,0,0,...,44,18,2,5,77,8,31,21,20,14
?|100134869,37,136,35,11,80,11,8,10,35,32,...,44,29,32,38,87,0,43,18,27,15
?|10357,161,274,707,33,585,296,136,355,218,566,...,347,283,127,212,388,11,487,236,235,185
?|10431,2669,1192,2306,521,3163,2567,2104,1832,3979,2813,...,3695,3331,2515,3736,3483,704,3623,4415,4629,3728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYX|7791,12281,8510,13045,1337,10031,4227,9223,9889,9247,14063,...,11651,7217,7165,7997,8640,10208,16541,11970,12247,7523
ZZEF1|23140,350,2790,1148,462,1538,1513,758,1488,822,1326,...,1927,927,1201,1328,3631,212,3439,1227,1530,1468
ZZZ3|26009,1267,1972,3455,383,1717,1690,286,1735,321,923,...,1576,977,724,742,2854,160,2243,748,1280,1062


In [6]:
# make countDat and colDat files for CIMP+ vs. CIMP-

classtmp = pd.read_csv("data/cimp_classes.csv", skiprows=1, index_col=0, header=None)
classtmp.index = [x.replace('.','-') for x in classtmp.index]
crcclasses = pd.concat([classtmp, pd.read_csv("data/cimp_classes.csv", index_col=0, header=None)], axis=0)
crcclasses = crcclasses[~crcclasses.index.duplicated(keep='first')] # drop duplicates
posnegclasses = crcclasses[(crcclasses[2] == "CIMP+") | (crcclasses[2] == "CIMP-")] # get CIMP+ and CIMP- samples

posnegexpr = expr[[x for x in expr.columns if x[:15] in posnegclasses.index]] # make countData
posnegmeta = pd.DataFrame(index=posnegexpr.columns,columns=["class"]) # make dataframe for metadata (colData)
for row in posnegmeta.index:
    posnegmeta.at[row,"class"] = posnegclasses.at[row[:15],2]
print("num CIMP+:",len(posnegmeta[posnegmeta["class"] == "CIMP+"]))
print("num CIMP-:",len(posnegmeta[posnegmeta["class"] == "CIMP-"]))
posnegmeta = posnegmeta.replace("CIMP+", "CIMPpositive") # replace class names as + and - characters aren't allowed
posnegmeta = posnegmeta.replace("CIMP-", "CIMPnegative")

posnegexpr.to_csv("output/countData-posneg.txt",sep="\t")
posnegmeta.to_csv("output/colData-posneg.txt",sep="\t")

posnegmeta

num CIMP+: 35
num CIMP-: 60


,class
TCGA-A5-A1OH-01A-21R-A22K-07,CIMPnegative
TCGA-A5-A2K2-01A-11R-A18M-07,CIMPnegative
TCGA-A5-A2K4-01A-11R-A18M-07,CIMPnegative
TCGA-A5-A2K5-01A-11R-A180-07,CIMPpositive
TCGA-A5-A2K7-01A-11R-A180-07,CIMPpositive
...,...
TCGA-PG-A5BC-01A-12R-A27V-07,CIMPnegative
TCGA-PG-A6IB-01A-21R-A31O-07,CIMPpositive
TCGA-QF-A5YS-01A-11R-A31O-07,CIMPnegative
TCGA-QF-A5YT-01A-11R-A31O-07,CIMPpositive


In [4]:
# make countDat and colDat files for MSI-H CIMP+ vs. MSI-H CIMP-

clindat = pd.read_csv("../raw-data/tcga-data/coadreadClinical.txt",sep='\t',index_col=0)
clindat.columns = [x.upper() for x in clindat.columns]
clindat = clindat.fillna("Unknown")
clindat = clindat.T
clindat = clindat[clindat["mononucleotide_and_dinucleotide_marker_panel_analysis_status"] == "msi-h"]
keep = [x for x in posnegexpr.columns if x[:12] in clindat.index]

msiposnegexpr = posnegexpr[keep]
msiposnegmeta = posnegmeta.loc[keep]
msiposnegmeta = msiposnegmeta.replace("CIMPpositive", "MSIH_CIMPpositive") # replace class names to include MSIH
msiposnegmeta = msiposnegmeta.replace("CIMPnegative", "MSIH_CIMPnegative")

msiposnegexpr.to_csv("output/countData-msiposneg.txt",sep="\t")
msiposnegmeta.to_csv("output/colData-msiposneg.txt",sep="\t")

clindat

FileNotFoundError: [Errno 2] No such file or directory: '../raw-data/tcga-data/coadreadClinical.txt'

In [4]:
msiposnegmeta

,class
TCGA-A6-2675-01A-02R-1723-07,MSIH_CIMPnegative
TCGA-A6-2682-01A-01R-1410-07,MSIH_CIMPnegative
TCGA-A6-2684-01A-01R-1410-07,MSIH_CIMPnegative
TCGA-A6-2685-01A-01R-1410-07,MSIH_CIMPnegative
TCGA-A6-2686-01A-01R-A32Z-07,CIMPpositive
...,...
TCGA-F5-6814-01A-31R-1928-07,CIMPpositive
TCGA-F5-6863-01A-11R-1928-07,MSIH_CIMPnegative
TCGA-G5-6233-01A-11R-1736-07,CIMPpositive
TCGA-G5-6572-01A-11R-1830-07,MSIH_CIMPnegative
